In [151]:
#Importamos librerias
import pandas as pd
import numpy as np
import re
import warnings

#Importamos funciones de nuestro script
from functions import *

In [152]:
#Importamos la data

url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'

shark_attack_raw = pd.read_excel(url)

shark_attack_raw

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,15 Mar 2024,2024.0,Unprovoked,AUSTRALIA,Queensland,Bargara Beach,Swimming,Brooklyn Sauer,F,13,...,Tiger shark,"Yahoo News, 3/15/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04 Mar 2024,2024.0,Unprovoked,USA,Hawaii,"Old Man's, Waikiki",Surfing,Matthew White,M,NaN,...,Tiger shark 8',"Surfer, 3/6/2024F",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02 Mar-2024,2024.0,Unprovoked,USA,Hawaii,"Rainbows, Oahu",Swimming,NaN,F,11,...,3' to 4' shark,"Hawaii News Now, 3/4/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25 Feb-2024,2024.0,Unprovoked,AUSTRALIA,Western Australia,"Sandlnd Island, Jurian Bay",NaN,female,F,46,...,Tiger shark,"WA Today, 2/26/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14 Feb-2024,2024.0,Unprovoked,INDIA,Maharashtra,"Vaitarna River, Palghar District",Fishing,Vicky Suresh Govari,M,32,...,"Bull shark, 7'","Times of India, 2/14/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN


### Bussines Target: rentabilizar la data(climatología, conservación del medio ambiente).

### Limpieza de datos

In [153]:
#LIMPIEZA DE DATOS
shark_attack_raw = clean_columns_names(shark_attack_raw, shark_attack_raw.columns)
#shark_attack_raw #comprobración
#LIMPIAMOS LAS COLUMNAS IRRELEVANTES PARA NUESTRO ESTUDIO
shark_attack_clean = columns_drops(shark_attack_raw, [ 'year', 'type', 'name', 'unnamed:_11', 'time', 'source', 'pdf', 'href_formula', 'href', 'case_number', 'case_number.1',
       'original_order', 'unnamed:_21', 'unnamed:_22'])
#shark_attack_raw.head(5)#comprobración



In [154]:
shark_attack_clean.nunique()

date        6003
country      225
state        902
location    4514
activity    1587
sex            8
age          243
injury      4079
species_    1675
dtype: int64

In [155]:
recuento_nulls = shark_attack_clean.isnull().sum()
recuento_nulls

date          25
country       75
state        507
location     590
activity     611
sex          604
age         3019
injury        60
species_    3157
dtype: int64

In [156]:
#TRATAMIENTO NULOS
shark_attack_clean = drop_rows_nulls(shark_attack_clean, thresh=2)
recuento_nulls = shark_attack_clean.isnull().sum()
recuento_nulls

date           0
country       50
state        482
location     565
activity     586
sex          579
age         2994
injury        35
species_    3132
dtype: int64

In [157]:
shark_attack_clean.shape
shark_attack_clean.info

<bound method DataFrame.info of              date             country              state  \
0     15 Mar 2024           AUSTRALIA         Queensland   
1     04 Mar 2024                 USA             Hawaii   
2     02 Mar-2024                 USA             Hawaii   
3     25 Feb-2024           AUSTRALIA  Western Australia   
4     14 Feb-2024               INDIA        Maharashtra   
...           ...                 ...                ...   
6939  Before 1903           AUSTRALIA  Western Australia   
6940  Before 1903           AUSTRALIA  Western Australia   
6941    1900-1905                 USA     North Carolina   
6942    1883-1889              PANAMA                NaN   
6943    1845-1853  CEYLON (SRI LANKA)   Eastern Province   

                                 location      activity sex  age  \
0                           Bargara Beach      Swimming   F   13   
1                      Old Man's, Waikiki       Surfing   M  NaN   
2                          Rainbows, Oahu  

In [158]:
recount_dupli = shark_attack_clean.duplicated().sum()
recount_dupli

10

In [159]:
shark_attack_clean.shape

(6944, 9)

In [160]:
#Eliminamos duplicados
shark_attack_clean.drop_duplicates(inplace=True)
shark_attack_clean.shape

(6934, 9)

In [162]:
shark_attack_clean  = shark_attack_clean.dropna(how='all')
recuento_nulls = shark_attack_clean.isnull().sum()
recuento_nulls

date           0
country       50
state        481
location     563
activity     583
sex          579
age         2984
injury        35
species_    3123
dtype: int64

In [163]:
shark_attack_clean.head()

,date,country,state,location,activity,sex,age,injury,species_
0,15 Mar 2024,AUSTRALIA,Queensland,Bargara Beach,Swimming,F,13,"Minor injuries to back, abdomen and legs",Tiger shark
1,04 Mar 2024,USA,Hawaii,"Old Man's, Waikiki",Surfing,M,NaN,"No injury, shark bit surfboard",Tiger shark 8'
2,02 Mar-2024,USA,Hawaii,"Rainbows, Oahu",Swimming,F,11,Lacerations to left foot,3' to 4' shark
3,25 Feb-2024,AUSTRALIA,Western Australia,"Sandlnd Island, Jurian Bay",NaN,F,46,Leg bitten,Tiger shark
4,14 Feb-2024,INDIA,Maharashtra,"Vaitarna River, Palghar District",Fishing,M,32,Calf of lower left leg injured,"Bull shark, 7'"


-----------------------------------------------------------------------------------------------------------------------------------------

## Valores

In [164]:
#Tratamiento de fechas
shark_attack_values = clean_and_filter_dates(shark_attack_clean, 'date', '1989')

In [166]:
shark_attack_values.dtypes

date        datetime64[ns]
country             object
state               object
location            object
activity            object
sex                 object
age                 object
injury              object
species_            object
dtype: object

In [ ]:
shark_attack_values['injuries'] = shark_attack_values['injury'].apply(categorize_injury)
shark_attack_values['injuries'] = shark_attack_values['injury'].apply(categorize_injury)

## Gráficos